# 투자전략 1: 모멘텀 투자 - S&P500

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import logging
from IPython.display import Image

In [4]:
pd.options.display.max_columns = 100 # was 20
pd.options.display.max_rows = 30 # was 60
logger = logging.getLogger('zero_remove_logger')

In [5]:
df = pd.read_csv("SP500_out_2015-2018.csv", encoding="UTF-8")

In [6]:
df.date = pd.to_datetime(df.date)

In [7]:
t = pd.Timedelta('180 days') # 180일 간격으로 window가 움직인다. 

In [10]:
class MovingWindow:
    def __init__(self):
        self.top100firms = []
        self.top100 = pd.DataFrame()
        self.holding = pd.DataFrame()
        self.window = pd.Timedelta('365 days')
        self.balance = 0
        self.total_investment = 1
        self.current_date = pd.Timestamp('2016-12-30 00:00:00') # 너무 많은 주식들이 2014.12.31엔 0원이다.2015도... # 중요! S&P500 돌릴땐 12.30으로 해야함. 2016.12.31이 공휴일이었나봐. 
    
    def get_top100(self):
        basket = pd.DataFrame()
        basket['firms'] = df.columns[1:]
        basket['ipo_index'] = df.drop(['date'], axis=1).ne(0).idxmax().values #마지막에 values 붙여줘야 series를 df에 column으로 부착 가능. 
        basket['ipo_price'] = 0
        
        for idx, row in basket.iterrows(): # 이 부분이 iterrows 때문에 병목이 생기는 것으로 보임. 
            basket['ipo_price'][idx] = df.drop(['date'], axis=1)[row["firms"]].iloc[row['ipo_index']]
        basket['current_price'] = df.drop(['date'], axis=1).loc[df['date']==self.current_date].squeeze().values
        
        basket = basket[basket.current_price > basket.ipo_price] # +수익률만 남김.
        
        basket['price_change'] = 100*(basket.current_price - basket.ipo_price)/basket.ipo_price
        basket = basket.sort_values(by='price_change', ascending=False) # 수익률 순으로 내림차순 정렬. 
        
        basket['holding'] = 100
        
        self.top100 = basket.head(100)
        self.holding = self.top100[['firms', 'holding']]
        self.top100firms = list(self.top100.firms)
        
        self.top100['TR'] = self.top100.current_price * self.top100.holding
        self.top100['TC'] = self.top100.current_price * self.top100.holding
        
        return self.top100
        
    def after_t_return(self):
        self.current_date += t
        basket = df[['date']+list(self.top100firms)]

        self.top100['firms'] = self.top100firms
        self.top100['current_price'] = basket[basket['date'] == self.current_date].drop(['date'], axis=1).squeeze().values
        
        self.top100['year_ago_price'] = basket[basket['date'] == (self.current_date - self.window)].drop(['date'], axis=1).squeeze().values
        
        for idx, row in self.top100.iterrows(): # t만큼 이동 후 1년 전의 시점에 아직 IPO가 안됐을 경우, 최초 IPO 가격으로 대체. 
            if row.year_ago_price == 0:
                self.top100.year_ago_price[idx] = row.ipo_price
                
        self.top100['price_change'] = 100*(self.top100.current_price - self.top100.year_ago_price)/self.top100.year_ago_price
        
        self.top100 = self.top100.sort_values(by='price_change', ascending=False) # 수익률 순으로 내림차순 정렬. 
        
        self.top100 = pd.merge(self.top100, self.holding, on=['firms'], suffixes=('_x', '')) #일단 그냥 중복 컬럼 생기게 두고, 지워버린다. 
        try:
            self.top100 = self.top100.drop(['holding_x'], axis=1)
            self.top100 = self.top100.drop(['current_price_x'], axis=1)
        except:
            pass
    
        self.top100.TR = self.top100.current_price * self.top100.holding
        
        self.top100['profit'] = self.top100.TR - self.top100.TC
        self.top100['return'] = 100*(self.top100.profit / self.top100.TC)
        
        self.balance = self.top100['profit'].sum()
        self.total_investment = self.top100['TC'].sum()
        
        return self.top100
    
    def buy_and_sell(self):                             
        buy_firms = self.top100.head(10)['firms'].values
        sell_firms = self.top100.tail(10)['firms'].values
        
        for f in buy_firms: # 중간에 매수한 비용은 해당 주식 매수 비용에 더함. 
            self.top100.loc[self.top100.firms == f, 'TC'] = self.top100.loc[self.top100.firms == f, 'TC'] + self.holding[self.holding.firms == f].holding.values[0] * 0.1 * self.top100.loc[self.top100.firms == f, 'current_price']
            self.holding.loc[self.holding.firms == f, 'holding'] = self.holding[self.holding.firms == f].holding.values[0] * 1.1 #(rebalancing에서 총 투자금액이 아닌)주식 수량을 10% 늘리는건 좀 문제가 있을 것 같지만 그냥 함. 
        
        for f in sell_firms: # 중간에 매도하여 얻은 차익은 balance에 더함. 
#             self.top100.loc[self.top100.firms == f, 'TC'] = self.top100.loc[self.top100.firms == f, 'TC'] - self.holding[self.holding.firms == f].holding.values[0] * 0.1 * self.top100.loc[self.top100.firms == f, 'current_price']
            self.balance += self.holding[self.holding.firms == f].holding.values[0] * 0.1 * self.top100.loc[self.top100.firms == f, 'current_price'] 
            self.holding.loc[self.holding.firms == f, 'holding'] = self.holding[self.holding.firms == f].holding.values[0] * 0.9 #이것도. 수량을 10% 줄임. 
        
        return self.holding
    
    def investment_return(self):
        return 100*(self.balance / self.total_investment)
        

In [15]:
def after_t_result(s):
    s.buy_and_sell()
    try:
        s.after_t_return()
    except:
        s.current_date = s.current_date - np.timedelta64(4,'D') - t
        s.after_t_return()
    
    print(s.current_date)
    print("net profit: ", s.balance)
    print("total investment: ", s.total_investment)
    print("investment return (%): ", s.investment_return())

## 투자 시작. 

In [12]:
s1 = MovingWindow()

### 1기: 주식 선택

#### KOSPI200 중 2016년 12월 31일 기준으로 1년 전 가격과 비교하여 (또는 그 기간 중 IPO 가격) 가장 가격 증가율이 높은 100개를 골라 포트폴리오를 구성한다. 

In [13]:
s1.get_top100()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,firms,ipo_index,ipo_price,current_price,price_change,holding,TR,TC
7,AMD.Adjusted,0,2,11.340000,467.000000,100,1134.0000,1134.0000
343,NVDA.Adjusted,0,19,106.135986,458.610453,100,10613.5986,10613.5986
3,ABMD.Adjusted,0,37,112.680000,204.540541,100,11268.0000,11268.0000
325,NFLX.Adjusted,0,49,123.800003,152.653067,100,12380.0003,12380.0003
29,AMZN.Adjusted,0,308,749.869995,143.464284,100,74986.9995,74986.9995
294,MLM.Adjusted,0,107,218.358368,104.073241,100,21835.8368,21835.8368
444,ULTA.Adjusted,0,126,254.940002,102.333335,100,25494.0002,25494.0002
466,VMC.Adjusted,0,65,123.250145,89.615608,100,12325.0145,12325.0145
5,ATVI.Adjusted,0,19,35.706181,87.927268,100,3570.6181,3570.6181
328,NEM.Adjusted,0,18,33.427715,85.709528,100,3342.7715,3342.7715


In [14]:
s1.current_date

Timestamp('2016-12-30 00:00:00')


### 2기: t(여기선 6개월)가 지난 후 100개의 포트폴리오를 재평가.

#### 이 시점에서 번 돈과(각 주식 profit의 총 합 = balance) 투자한 총 금액(각 주식 Total Cost의 총 합 = total investment), 그리고 전체 투자의 수익률을 확인한다. (%) 

In [16]:
after_t_result(s1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2017-06-28 00:00:00
net profit:  180875.11555000005
total investment:  1068746.00645
investment return (%):  16.924050659221066



### 3기: t(여기선 6개월)가 지난 후 100개의 포트폴리오를 재평가.

#### 이 시점에서 번 돈과(각 주식 profit의 총 합 = balance) 투자한 총 금액(각 주식 Total Cost의 총 합 = total investment), 그리고 전체 투자의 수익률을 확인한다. (%) 

In [17]:
after_t_result(s1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2017-12-21 00:00:00
net profit:  340439.30173600005
total investment:  1079240.288439
investment return (%):  31.544347017327095



### 4기: t(여기선 6개월)가 지난 후 100개의 포트폴리오를 재평가.

#### 이 시점에서 번 돈과(각 주식 profit의 총 합 = balance) 투자한 총 금액(각 주식 Total Cost의 총 합 = total investment), 그리고 전체 투자의 수익률을 확인한다. (%) 

In [18]:
after_t_result(s1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2018-06-19 00:00:00
net profit:  578088.3936276003
total investment:  1095160.446132
investment return (%):  52.785726116146
